In [34]:
import pickle
import pandas as pd

In [20]:
source.update

<bound method HasProps.update of ColumnDataSource(id='789cd1c7-91c5-403c-aa9b-d130df82a0ae', ...)>

In [21]:
[model for model in city_dict[city].columns]

NameError: name 'city_dict' is not defined

In [99]:
source.column_names

['Boston, MA metro area All Residential Median Sale Price Trend Seasonality Additional',
 'Boston, MA metro area All Residential Median Sale Price',
 'Boston, MA metro area All Residential Median Sale Price Trend Seasonality',
 'Boston, MA metro area All Residential Median Sale Price Baseline Trend',
 'Boston, MA metro area All Residential Median Sale Price Best ARMA',
 'Boston, MA metro area All Residential Median Sale Price Best ARMA Add',
 'Boston, MA metro area All Residential Median Sale Price Best SARMA',
 'index',
 'y']

 # new

In [23]:
#!pip install --upgrade pip

In [24]:
from bokeh.io import curdoc 
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import row
from bokeh.io import gridplot, output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.models.widgets import Dropdown
from bokeh.models.widgets import Select
from bokeh.layouts import widgetbox

#output_file("layout.html")
import pandas as pd
import re
import datetime

In [25]:
df = pd.read_csv('./dsifolder_endofclass/bos_bokeh.csv')#, index_col=0)
df['date'] = pd.to_datetime(df['Unnamed: 0'])
df = df.drop('Unnamed: 0',axis=1)

pred_cols = list(df.columns)
pred_cols = pred_cols[1:7]
error_cols = [col for col in df.columns if re.findall('Error',col)]

In [27]:
menu_val = models
cds = pd.DataFrame()
model_pred_cols = []
model_error_cols = []
model_legs = []
models = ['Best ARMA','Best ARMA Add','Best SARMA','Baseline Trend','Trend Seasonality','Trend Seasonality Additional']
for model,pred_col,error_col in zip(models,pred_cols,error_cols): 
    if model in menu_val:
        model_pred_cols.append(pred_col)
        model_error_cols.append(error_col)
        model_legs.append(model)
cds = pd.concat([df[model_pred_cols],df[model_error_cols]],axis=1)
cds = pd.concat([cds,df['date']],axis=1)
cds = pd.concat([cds,df['Boston, MA metro area All Residential Median Sale Price']],axis=1)

In [28]:
model_pred_cols

['Boston, MA metro area All Residential Median Sale Price Best ARMA',
 'Boston, MA metro area All Residential Median Sale Price Best ARMA Add',
 'Boston, MA metro area All Residential Median Sale Price Best SARMA',
 'Boston, MA metro area All Residential Median Sale Price Baseline Trend',
 'Boston, MA metro area All Residential Median Sale Price Trend Seasonality',
 'Boston, MA metro area All Residential Median Sale Price Trend Seasonality Additional']

In [29]:
source = ColumnDataSource(cds)    

In [30]:
    
# source = ColumnDataSource(
#         data=dict(
#             a = df['Boston, MA metro area All Residential Median Sale Price'],
#             b = df['Boston, MA metro area All Residential Median Sale Price Best ARMA'],
#             x = df['date']
#         )
#     )


In [31]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>





In [32]:
%%javascript 
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [33]:
from bokeh.charts import TimeSeries
from bokeh.models import ColumnDataSource, DataRange1d, Range1d
from bokeh.io import gridplot, output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.models.widgets import Dropdown
from bokeh.models.widgets import Select
from bokeh.layouts import widgetbox
from bokeh.palettes import Spectral7, Dark2_7, Category10, Viridis7, Category10_7, Set2_7
from bokeh.models import Legend
output_notebook()

# city
city = 'Boston, MA metro area'

#top line chart y_range=Range1d(300, 510)
p = figure(plot_width=900, plot_height=350, title=city+' Predictions', x_axis_type="datetime")
for model_pred_col,leg,color in zip(model_pred_cols,model_legs,Dark2_7):
    #print(model_pred_col)
    p.line('date',model_pred_col, source=source, legend=leg, color=color, line_width=2)
#draw the target
p.line('date','Boston, MA metro area All Residential Median Sale Price', 
       source=source, legend='Median Sale Price',color='red', line_width=2)


ydr = DataRange1d()

#bottom chart
q = figure(plot_width=900, plot_height=350, x_range=p.x_range, y_range=Range1d(-30, 30), title=city+' Errors', x_axis_type="datetime")
for model_error_col,leg,color in zip(model_error_cols,model_legs,Dark2_7):
    q.vbar('date', top=model_error_col, width=.5, source=source, legend=leg, color=color)

#p.line(,'Boston, MA metro area All Residential Median Sale Price Trend Seasonality Additional Error')
#p.circle('x', 'a', size=4, source=source)

 #main plot
p.background_fill_color = "gray"
p.background_fill_alpha = .2

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = 'black'
p.ygrid.grid_line_alpha = 0.5
p.ygrid.grid_line_dash = [6, 4]

q.background_fill_color = "gray"
q.background_fill_alpha = .2

# q.xgrid.grid_line_color = None
# q.ygrid.grid_line_color = 'black'
# q.ygrid.grid_line_alpha = 0.5
# q.ygrid.grid_line_dash = [6, 4]


#move legend outside
# legend = Legend((items=models), location=(0, -30))

# p.add_layout(legend, 'right')

# put all the plots in a grid layout
r = gridplot([[p], [q]])

p.legend.location = "top_left"
p.legend.click_policy = "hide"

q.legend.location = "top_left"
q.legend.click_policy = "hide"

output_notebook()

show(r)

Loading BokehJS ...

Loading BokehJS ...

# What models predict housing prices the best, and why?

<img src="/files/seasonality.png" style="float: left; height: 510px">
Redfin housing data of ~30 major metro regions in the USA

# Key Findings:
- Each housing market has a unique seasonal trends
    - Assuming the quality of housing stock is equal throughout the season, it is key to base purchasing decisions off of the local market trend, not the national
        - __Boston__, buy in _Feb_, sell _July through August_, range of __65k__
        - __San Jose__, buy in _Jan_, sell _April through July_, range of __100k__
- Seasonal Decomposition models perform best across all cities
    - 57% better than trend baseline
    - 27-56% better than best SARIMA
    - 37-52% better than best ARIMA


<img src="/files/seasonality.png" style="float: left; height: 510px">

<img src="/files/hists_t1.png" style="float: left; height: 800px">

<img src="/files/a.png" style="float: left; height: 150px">